In [ ]:
#====LISTINGS NEW YORK CLEANER ====


import pandas as pd
import re

# === 1. Read your original CSV ===
# Adjust the path if needed
input_file = "new_york_listings_2024.csv"
output_file = r"C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\new_york_listings_fixed.csv"


# Try reading with comma separator (change sep=';' if your file uses semicolons)
df = pd.read_csv(r"C:\Users\elbgr\OneDrive\Escritorio\housing_nyc\new_york_listings_2024.csv", sep=",", encoding="utf-8", dtype=str)

# === 2. Clean latitude and longitude ===
def clean_number(value):
    if pd.isna(value):
        return None
    # Remove all non-numeric symbols except minus and comma/dot
    v = re.sub(r"[^0-9,.\-]", "", str(value))
    # Remove thousand separators: replace dots between digits with nothing
    v = re.sub(r"(?<=\d)\.(?=\d{3}\b)", "", v)
    # Replace comma with dot for decimal
    v = v.replace(",", ".")
    try:
        return float(v)
    except ValueError:
        return None

df["latitude"] = df["latitude"].apply(clean_number)
df["longitude"] = df["longitude"].apply(clean_number)

df_cleaned = df.drop(columns=['number_of_reviews_ltm', 'license', 'rating', 'bedrooms', 'beds', 'baths'])

df_cleaned=df.drop(rows)

df.head()



#3. Save cleaned file 
#df_cleaned.to_csv(output_file, index=False, encoding="utf-8", sep=",", quoting=1)

#print(f"✅ Clean CSV saved as: {output_file}")





✅ Clean CSV saved as: C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\new_york_listings_fixed.csv


In [ ]:
#====FILTER INVALID LATITUDE AND LONGITUDE ====


import pandas as pd

in_file  = r"C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\new_york_listings_fixed.csv"
out_file = r"C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\new_york_listings_clean.csv"

df = pd.read_csv(in_file, encoding="utf-8")

# Convertir a número (sin alterar el texto original)
df["lat_num"] = pd.to_numeric(df["latitude"], errors="coerce")
df["lon_num"] = pd.to_numeric(df["longitude"], errors="coerce")

# --- Filtro por rango y magnitud razonable ---
mask = (
    df["lat_num"].between(40, 41) &   # latitudes alrededor de NYC
    df["lon_num"].between(-75, -72)   # longitudes razonables
)

# Aplicar filtro
df_clean = df[mask].copy()
df_clean.drop(columns=["lat_num", "lon_num"], inplace=True)

# Guardar
df_clean.to_csv(out_file, index=False, encoding="utf-8")

print(f"Filas originales: {len(df)}")
print(f"Filas válidas: {len(df_clean)}")
print(f"Archivo limpio guardado en: {out_file}")




Filas originales: 20750
Filas válidas: 20480
Archivo limpio guardado en: C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\new_york_listings_clean.csv


In [ ]:
import pandas as pd
import re
import csv
from pathlib import Path

# === RUTAS: cámbialas a las tuyas ===
INPUT  = r"C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\new_york_housing.csv"
OUTPUT = r"C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\new_york_housing_fixed.csv"

print("Leyendo:", INPUT)

# 1) Leer robusto
df = pd.read_csv(INPUT, sep=",", encoding="utf-8", engine="python", on_bad_lines="skip")
df.columns = [c.strip().upper() for c in df.columns]

print("Shape original:", df.shape)

# 2) Limpieza numérica básica
def clean_number(x):
    if pd.isna(x): 
        return None
    v = str(x)
    v = re.sub(r"[^0-9,\.\-]", "", v)                 # deja dígitos, coma, punto y signo
    v = re.sub(r"(?<=\d)[\.\s](?=\d{3}\b)", "", v)    # quita separadores de miles 739.744.834 -> 739744834
    v = v.replace(",", ".")                           # coma decimal -> punto
    try:
        return float(v)
    except ValueError:
        return None

def fix_coord(x, kind):
    """
    kind = 'lat' o 'lon'
    - Convierte a float con clean_number
    - Si queda fuera de rango, intenta reescalar dividiendo por 10 hasta caer en rango
      (caso típico: -739744834 -> -73.9744834)
    """
    v = clean_number(x)
    if v is None:
        return None
    lim = 90 if kind == "lat" else 180
    # Reescala si está fuera de rango por miles erróneos
    tries = 0
    while abs(v) > lim and tries < 10:
        v = v / 10.0
        tries += 1
    # Si aún está fuera de rango, descártala
    if abs(v) > lim:
        return None
    return round(v, 6)

before_nulls = {}
for col in ("LATITUDE", "LONGITUDE"):
    if col in df.columns:
        before_nulls[col] = df[col].isna().sum()

if "LATITUDE" in df.columns:
    df["LATITUDE"] = df["LATITUDE"].apply(lambda x: fix_coord(x, "lat"))
if "LONGITUDE" in df.columns:
    df["LONGITUDE"] = df["LONGITUDE"].apply(lambda x: fix_coord(x, "lon"))

after_nulls = {}
for col in ("LATITUDE", "LONGITUDE"):
    if col in df.columns:
        after_nulls[col] = df[col].isna().sum()

print("Nulos antes:", before_nulls)
print("Nulos después:", after_nulls)

# NO elimino todas las filas por rango; solo quito las que sigan fuera de rango claro
if "LATITUDE" in df.columns and "LONGITUDE" in df.columns:
    valid = df["LATITUDE"].between(-90, 90, inclusive="both") & df["LONGITUDE"].between(-180, 180, inclusive="both")
    kept = valid.sum()
    dropped = len(df) - kept
    print(f"Filas válidas por rango lat/lon: {kept}  |  Filas descartadas: {dropped}")
    df = df[valid]

print("Shape final:", df.shape)

# 3) Guardar amigable para MySQL
df.to_csv(
    OUTPUT,
    index=False,
    encoding="utf-8",
    lineterminator="\n",
    quoting=csv.QUOTE_MINIMAL,   # usa QUOTE_ALL si prefieres todo entrecomillado
    quotechar='"',
    escapechar='\\'
)

print("Guardado en:", OUTPUT)



Leyendo: C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\new_york_housing.csv
Shape original: (4801, 17)
Nulos antes: {'LATITUDE': np.int64(0), 'LONGITUDE': np.int64(0)}
Nulos después: {'LATITUDE': np.int64(0), 'LONGITUDE': np.int64(0)}
Filas válidas por rango lat/lon: 4801  |  Filas descartadas: 0
Shape final: (4801, 17)
Guardado en: C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\new_york_housing_fixed.csv
